# Information Retrieval - Phase 2: Query Expansion with WordNet Synonyms on the IR2025 Collection


In this project, we expand IR2025 queries using **WordNet synonyms** (via **NLTK**), re-run our **BM25 Elasticsearch** retrieval, and evaluate the impact of synonym-based query expansion.

---
> Maria Schoinaki, BSc Student <br />
> Department of Informatics, Athens University of Economics and Business <br />
> p3210191@aueb.gr <br/><br/>

> Nikos Mitsakis, BSc Student <br />
> Department of Informatics, Athens University of Economics and Business <br />
> p3210122@aueb.gr <br/><br/>

_In this notebook, we will:_

1. Set up the Python environment and Elasticsearch connection  
2. Load original queries, relevance judgments (qrels), and prepare the TF–IDF model  
3. Define POS-mapping and synonym-extraction functions using NLTK’s WordNet API  
4. Generate **Scenario A** expanded queries (all single-word synonyms) and write them to `queries_expanded_wordnet.jsonl`  
5. Generate **Scenario B** expanded queries (hypernym-only) and write them to `queries_expanded_hypernym.jsonl`  
6. Execute top-k retrieval (k = 20, 30, 50) for both Scenario A and Scenario B  
7. Evaluate retrieval performance with MAP@k and P@k (k = 5, 10, 15, 20) via `pytrec_eval` for each scenario  
8. Compare Phase 2 results (both scenarios) against the Phase 1 BM25 baseline  

### Start ElasticSearch manually before running the notebook:
On Windows:
- Make sure you have at least JDK 17
- Open a terminal and execute this (or run it as a Windows service):
```bash
C:\path\to\elasticsearch-8.17.2\bin\elasticsearch.bat
```
- No Greek characters should be present in the path.
- Leave that terminal window open.

- If no password was autogenerated execute this to get one:
```bash
.\bin\elasticsearch-reset-password.bat -u elastic
```

In [1]:
%pip install -qq -r "..\\requirements.txt" 
# fix path accordingly

Note: you may need to restart the kernel to use updated packages.


## 1. Setup & Imports  

**3210122 + 3210191 = 6420313**
- So we get the `trec_covid` IR2025 collection.

Here we import all necessary libraries, set up environment variables, and instantiate the Elasticsearch client.

In [2]:
from collections import Counter
import jsonlines
import json
import csv
import pandas as pd
from tqdm import tqdm
import pytrec_eval
from IPython.display import display

Configuration & Parameters

In [3]:
from dotenv import load_dotenv
import os

# Load .env file from the current directory
load_dotenv("..\\secrets\\secrets.env")

# Access environment variables
es_host = os.getenv("ES_HOST")
es_user = os.getenv("ES_USERNAME")
es_pass = os.getenv("ES_PASSWORD")

Connect to ElasticSearch

In [4]:
from elasticsearch import Elasticsearch

es = Elasticsearch(es_host, basic_auth=(es_user, es_pass), request_timeout=30, retry_on_timeout=True, max_retries=10)

if es.ping():
    print("✅ Connected to ElasticSearch")
else:
    print("❌ Connection failed")

✅ Connected to ElasticSearch


## 2. Analyzer & Index Creation  
We define a **custom English analyzer** (standard tokenizer + lowercase + stopword removal + Krovetz stemming) and create the Elasticsearch index with BM25 similarity.

In [5]:
INDEX_NAME = "ir2025-index"

# Delete the index if it already exists
if es.indices.exists(index=INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
    print(f"✅ Index '{INDEX_NAME}' deleted")

# Define the settings and mappings for the index
settings = {
    "analysis": {
        "filter": {
            "english_stop": {
                "type": "stop",
                "stopwords": "_english_"
            },
            "english_stemmer": {
                "type": "kstem"
            }
        },
        "analyzer": {
            "custom_english": {
                "type": "custom",
                "tokenizer": "standard",
                "filter": [
                    "lowercase", # Converts all terms to lowercase
                    "english_stop", # Removes English stop words
                    "english_stemmer" # Reduces words to their root form usign kstem
                ]
            }
        }
    }
}

mappings = {
    "properties": {
        "doc_id": {"type": "keyword"},
        "text": {
            "type": "text",
            "analyzer": "custom_english",
            "similarity": "BM25"
        }
    }
}

# Create the index with the specified settings and mappings
es.indices.create(
    index=INDEX_NAME,
    settings=settings,
    mappings=mappings
)
print(f"✅ Index '{INDEX_NAME}' created")

✅ Index 'ir2025-index' deleted
✅ Index 'ir2025-index' created


## 3. Document Ingestion  
Using the `streaming_bulk` helper, we ingest all IR2025 documents in chunks of 500.  
A progress bar (tqdm) provides real‐time feedback on indexing throughput.

In [6]:
from elasticsearch.helpers import streaming_bulk

# Generator function to yield documents
def generate_documents(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            doc = json.loads(line)
            yield {
                "_index": INDEX_NAME,
                "_id": doc["_id"],
                "_source": {
                    "doc_id": doc["_id"],
                    "text": doc["text"]
                }
            }

# Count the total number of documents for the progress bar
with open("../data/trec-covid/corpus.jsonl", 'r', encoding='utf-8') as f:
    total_docs = sum(1 for _ in f)

# Initialize the progress bar
progress = tqdm(unit="docs", total=total_docs)

successes = 0
for ok, action in streaming_bulk(client=es, actions=generate_documents("../data/trec-covid/corpus.jsonl"), chunk_size=500):
    progress.update(1)
    successes += int(ok)

progress.close()
print(f"✅ Indexed {successes}/{total_docs} documents into '{INDEX_NAME}'")

100%|██████████| 171332/171332 [00:47<00:00, 3624.57docs/s]

✅ Indexed 171332/171332 documents into 'ir2025-index'


## 4. NLTK Setup and Corpus Preprocessing

Download required NLTK data, load the IR2025 corpus into memory, and define a Python function to simulate the `custom_english` analyzer for downstream TF–IDF modeling.

We download the necessary NLTK corpora and models.

- **Tokenization & POS Tagging**: `punkt_tab`, `averaged_perceptron_tagger`  
- **Stopword List**: `stopwords`  
- **Lexical Database**: `wordnet` and the multilingual WordNet (`omw-1.4`)

In [7]:
import nltk
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\mitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\mitsa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

We load the IR2025 corpus into memory as a list of JSON objects using `jsonlines.open`, preparing it for TF–IDF vectorization and analysis.

In [8]:
with jsonlines.open('../data/trec-covid/corpus.jsonl') as reader:
    corpus = [obj for obj in reader]

Simulate `custom_english` Analyzer in Python

Here we replicate our Elasticsearch `custom_english` analyzer pipeline in pure Python using NLTK:

1. **Lowercase & Trim**  
2. **Punctuation Removal**  
3. **Tokenization** (`word_tokenize`)  
4. **Stopword Filtering** (`stopwords.words('english')`)  
5. **Stemming** (PorterStemmer as a proxy for Krovetz)  

This function lets us preprocess text identically before building the TF–IDF model.

In [9]:
# Simulate custom_english Analyzer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer # KrovetzStemmer supports up to python 3.10 at best 
import string

# Initialize NLTK components
stop_words = set(stopwords.words('english'))

stemmer = PorterStemmer() # It's "Closer" to Korvetz than Snowball is

def es_like_preprocess(text):
    # Lowercase the text
    text = text.lower().strip()
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize the text
    tokens = word_tokenize(text)
    # Remove stopwords and apply stemming (Porter)
    processed_tokens = [stemmer.stem(token) for token in tokens if token not in stop_words or not token.isalpha()]
    # Join tokens back into a single string
    return ' '.join(processed_tokens)

---
# Query Expansion with Wordrnet synonyms

## 5. TF–IDF Model Construction and Persistence

In this section, we build a TF–IDF model over the preprocessed IR2025 corpus, compute key statistics, and save all artifacts for later use:

- **Preprocessing Statistics**: total tokens, unique tokens, average tokens per document  
- **TF–IDF Statistics**: vocabulary size, average/min/max IDF  
- **Model Artifacts**:  
  - `tfidf_vectorizer.joblib` (fitted `TfidfVectorizer`)  
  - `idf_scores.json` (term → IDF)  
  - `tfidf_statistics.json` (all collected statistics)

We implement three functions:

1. `build_and_save_tfidf_model(corpus, output_dir)`  
   - Preprocesses each document, updates statistics  
   - Fits `TfidfVectorizer`, extracts IDF scores  
   - Saves model and statistics to disk  

2. `load_tfidf_model(output_dir)`  
   - Loads the saved vectorizer, IDF scores, and statistics  
   - Prints summary for validation  

3. `transform_text(text, vectorizer)`  
   - Applies the loaded vectorizer to new text, returning its TF–IDF representation  


In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np
import joblib
import json
import os

def build_and_save_tfidf_model(corpus, output_dir="../models"):
    """
    Build TF-IDF model from corpus, compute statistics, and save the model.
    
    Args:
        corpus: List of documents with 'text' field
        output_dir: Directory to save models and statistics
    
    Returns:
        tuple: (vectorizer, idf_scores, statistics_dict)
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize counters for statistics
    total_tokens = 0
    unique_tokens = set()
    statistics = {}

    # Preprocess with detailed statistics
    print("Preprocessing corpus...")
    preprocessed_corpus = []
    for doc in tqdm(corpus, desc="Preprocessing documents", unit="doc"):
        processed_text = es_like_preprocess(doc["text"])
        tokens = processed_text.split()
        
        # Update statistics
        total_tokens += len(tokens)
        unique_tokens.update(tokens)
        
        preprocessed_corpus.append(processed_text)

    # Save preprocessing statistics
    statistics['preprocessing'] = {
        'total_tokens': total_tokens,
        'unique_tokens': len(unique_tokens),
        'average_tokens_per_doc': total_tokens/len(corpus)
    }

    print(f"\nPreprocessing statistics:")
    print(f"- Total tokens: {total_tokens:,}")
    print(f"- Unique tokens: {len(unique_tokens):,}")
    print(f"- Average tokens per document: {total_tokens/len(corpus):,.1f}")

    # Build TF-IDF model with detailed progress
    print("\nBuilding TF-IDF model...")
    tfidf_vectorizer = TfidfVectorizer(lowercase=True, stop_words='english')

    with tqdm(total=3, desc="TF-IDF computation") as pbar:
        # Fit the vectorizer
        tfidf_vectorizer.fit(preprocessed_corpus)
        pbar.update(1)
        
        # Get feature names
        feature_names = tfidf_vectorizer.get_feature_names_out()
        pbar.update(1)
        
        # Calculate IDF scores
        idf_scores = dict(zip(feature_names, tfidf_vectorizer.idf_))
        pbar.update(1)

    # Calculate and save TF-IDF statistics
    idf_values = list(idf_scores.values())
    statistics['tfidf'] = {
        'vocabulary_size': len(idf_scores),
        'average_idf': float(np.mean(idf_values)),
        'max_idf': float(max(idf_values)),
        'min_idf': float(min(idf_values))
    }

    print("\nTF-IDF statistics:")
    print(f"- Vocabulary size: {len(idf_scores):,} terms")
    print(f"- Average IDF: {statistics['tfidf']['average_idf']:.2f}")
    print(f"- Max IDF: {statistics['tfidf']['max_idf']:.2f}")
    print(f"- Min IDF: {statistics['tfidf']['min_idf']:.2f}")

    # Save everything
    print("\nSaving models and statistics...")
    try:
        # Save vectorizer
        joblib.dump(tfidf_vectorizer, os.path.join(output_dir, 'tfidf_vectorizer.joblib'))
        
        # Save IDF scores
        with open(os.path.join(output_dir, 'idf_scores.json'), 'w', encoding='utf-8') as f:
            json.dump(idf_scores, f, ensure_ascii=False, indent=2)
            
        # Save statistics
        with open(os.path.join(output_dir, 'tfidf_statistics.json'), 'w', encoding='utf-8') as f:
            json.dump(statistics, f, ensure_ascii=False, indent=2)
            
        print("\n✅ Saved successfully:")
        print(f"- Vectorizer: {os.path.join(output_dir, 'tfidf_vectorizer.joblib')}")
        print(f"- IDF scores: {os.path.join(output_dir, 'idf_scores.json')}")
        print(f"- Statistics: {os.path.join(output_dir, 'tfidf_statistics.json')}")
        
    except Exception as e:
        print(f"\n❌ Error saving files: {e}")
        
    return tfidf_vectorizer, idf_scores

# Function to load the saved model
def load_tfidf_model(output_dir="../models"):
    """
    Load the saved TF-IDF model, IDF scores, and statistics.
    
    Args:
        output_dir: Directory where models and statistics are saved
        
    Returns:
        tuple: (vectorizer, idf_scores, statistics)
    """
    try:
        # Load vectorizer
        vectorizer = joblib.load(os.path.join(output_dir, 'tfidf_vectorizer.joblib'))
        
        # Load IDF scores
        with open(os.path.join(output_dir, 'idf_scores.json'), 'r', encoding='utf-8') as f:
            idf_scores = json.load(f)
            
        # Load statistics
        with open(os.path.join(output_dir, 'tfidf_statistics.json'), 'r', encoding='utf-8') as f:
            statistics = json.load(f)
        
        print("\nModel validation:")
        print(f"- Vocabulary size: {len(idf_scores):,}")
        print(f"- Average IDF: {statistics['tfidf']['average_idf']:.2f}")
        print(f"- Max IDF: {statistics['tfidf']['max_idf']:.2f}")
        print(f"- Min IDF: {statistics['tfidf']['min_idf']:.2f}")
 
        print("✅ Model loaded successfully")
        return vectorizer, idf_scores
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None, None
        
# Transform new text using the loaded vectorizer
def transform_text(text, vectorizer):
    """Transform new text using the loaded vectorizer"""
    try:
        transformed = vectorizer.transform([text])
        return transformed
    except Exception as e:
        print(f"❌ Error transforming text: {e}")
        return None

## 6. Load or Build TF–IDF Model  

Attempt to load the saved TF–IDF vectorizer and IDF scores. If they are not found, build the model from the corpus and save the artifacts for future runs.

In [11]:
# Test loading
vectorizer, idf_scores = load_tfidf_model()
if not vectorizer and not idf_scores:
    # Build and save the model
    vectorizer, idf_scores = build_and_save_tfidf_model(corpus)


Model validation:
- Vocabulary size: 298,422
- Average IDF: 11.84
- Max IDF: 12.36
- Min IDF: 2.03
✅ Model loaded successfully


## 7. WordNet Synonym Extraction Function

Define a helper function `get_wordnet_synonyms` that:
- Queries all WordNet synsets for a given word  
- Extracts single-word, alphabetic lemmas excluding the original term  
- Ranks candidates by usage frequency (lemma count)  
- Returns up to `max_synonyms` most frequent synonyms  

In [12]:
from nltk.corpus import wordnet as wn
def get_wordnet_synonyms(word, max_synonyms=3):
    synonyms = set()
    word = word.lower()

    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            name = lemma.name().replace("_", " ").lower()

            # Filter out:
            if name == word: # Skip the original word
                continue
            if len(name.split()) > 1:  # Skip multi-word phrases
                continue
            if not name.isalpha(): # Skip non-alphabetic words
                continue

            synonyms.add(name)

    # Rank by frequency (most-used synonyms first)
    ranked_synonyms = sorted(synonyms, key=lambda s: -sum(lemma.count() for syn in wn.synsets(s) for lemma in syn.lemmas() if lemma.name().lower() == s))

    return ranked_synonyms[:max_synonyms] # Return up to max_synonyms

## 8. Query Expansion with WordNet and IDF-Based Term Selection

This function `expand_query_with_wordnet` performs controlled query expansion by:

1. **Tokenization & POS Tagging**  
   - Lowercases and tokenizes the input query  
   - Tags tokens and selects only nouns (NN*) and adjectives (JJ*)  

2. **Candidate Scoring by IDF**  
   - Looks up each candidate’s IDF weight via the fitted TF–IDF vectorizer  
   - Sorts candidates by descending IDF to prioritize rare, discriminative terms  
   - Keeps the top `n_expand` terms for expansion  

3. **Synonym Extraction & Filtering**  
   - Calls `get_wordnet_synonyms` to retrieve up to `max_synonyms` per selected term  
   - Filters out any synonyms already present in the original query  

4. **Query Assembly**  
   - Appends the chosen synonyms to the original query text  
   - Returns the expanded query string (Elasticsearch will re-analyze it)

In [13]:
from nltk import pos_tag

def is_expandable(pos):
    return pos.startswith('NN') or pos.startswith('JJ')  # nouns & adjectives
    
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def expand_query_with_wordnet(query_text, tfidf_vectorizer, max_synonyms=1, n_expand=1):
    
    # For expansion decisions, work with the original query text
    tokens = word_tokenize(query_text.lower())
    tagged = pos_tag(tokens)
    
    # Set for fast lookup
    original_words = set(tokens)
    
    # Candidate words = noun/adjective, not stopword, alphabetic
    candidates = [
        (word, pos) for word, pos in tagged
        if word.isalpha() and word not in stop_words and is_expandable(pos)
    ]

    # Get IDF scores from vectorizer to identify important terms
    # We need to preprocess the word to match vectorizer's vocabulary
    idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))
    
    # Score each candidate by IDF from the corpus
    scored = [
        (word, idf_scores.get(es_like_preprocess(word), 0.0))  # preprocess just for vocabulary lookup
        for word, _ in candidates
    ]

    # Sort by IDF weight descending and keep top-n
    top_words = [word for word, _ in sorted(scored, key=lambda x: -x[1])[:n_expand]]
    
    # Expand only top words
    expanded_terms = []
    for word in top_words:
        synonyms = get_wordnet_synonyms(word, max_synonyms) # Get more, filter later
        for syn in synonyms:
            if syn in original_words:
                continue  # Skip if already in query
            expanded_terms.append(syn)
            if len(expanded_terms) >= max_synonyms:
                break  # limit per word

        # Return original query + expansion terms (let Elasticsearch handle preprocessing)
    return query_text + " " + " ".join(expanded_terms)

Read the original IR2025 query set from the JSONL file into a list for subsequent expansion and retrieval.

In [14]:
with jsonlines.open('../data/trec-covid/queries.jsonl') as reader:
    queries = [obj for obj in reader]
    print(f"Loaded {len(queries)} queries.")

Loaded 50 queries.


Apply the `expand_query_with_wordnet` function to each original query, adding an `"expanded_text"` field to store the expanded query string.

In [15]:
expanded_queries = []
print("Expanding Queries..")
for query in tqdm(queries, unit="query"):
    new_query = query.copy()
    new_query["expanded_text"] = expand_query_with_wordnet(query["text"], vectorizer)
    expanded_queries.append(new_query)

Expanding Queries..


100%|██████████| 50/50 [00:35<00:00,  1.41query/s]


Write the list of expanded queries (including `"query_id"`, original `"text"`, and `"expanded_text"`) to a new JSONL file for later retrieval experiments.

In [16]:
with jsonlines.open("../data/trec-covid/queries_expanded_wordnet.jsonl", mode='w') as writer:
    for q in expanded_queries:
        writer.write(q)
    print("✅ Expanded queries saved to ../data/trec-covid/queries_expanded_wordnet.jsonl")

✅ Expanded queries saved to ../data/trec-covid/queries_expanded_wordnet.jsonl


---
## Query Expansion with hypernet replacement

Fetches synonyms of hypernyms (i.e., "parent" concepts) for a given word using WordNet, up to a specified tree depth.

In [17]:
def get_wordnet_hypernym_synonyms(word, max_synonyms=1, depth=1):
    """
    Get synonyms of hypernyms for a given word.
    
    Args:
        word: Target word.
        max_synonyms: Max number of terms to return.
        depth: How deep to go in the hypernym tree (1 = immediate parents).
        
    Returns:
        A ranked list of synonym candidates.
    """
    word = word.lower()
    synonyms = set()

    for syn in wn.synsets(word):
        # Explore hypernyms up to the given depth
        current_level = [syn]
        for _ in range(depth):
            next_level = []
            for s in current_level:
                next_level.extend(s.hypernyms())
            current_level = next_level

        # Collect lemma names from hypernyms
        for hyper in current_level:
            for lemma in hyper.lemmas():
                name = lemma.name().replace("_", " ").lower()
                if name != word and name.isalpha() and len(name.split()) == 1:
                    synonyms.add(name)

    # Rank by frequency
    ranked = sorted(synonyms, key=lambda s: -sum(lemma.count() for syn in wn.synsets(s) for lemma in syn.lemmas() if lemma.name().lower() == s))
    return ranked[:max_synonyms]

Expands a search query by adding hypernyms (parent concepts) of the most "important" words in the query, ranked using TF-IDF scores.

In [18]:
def expand_query_with_hypernyms(query_text, tfidf_vectorizer, max_synonyms=1, n_expand=1, depth=1):
    tokens = word_tokenize(query_text.lower())
    tagged = pos_tag(tokens)
    original_words = set(tokens)

    candidates = [
        (word, pos) for word, pos in tagged
        if word.isalpha() and word not in stop_words and is_expandable(pos)
    ]

    idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))
    
    scored = [
        (word, idf_scores.get(es_like_preprocess(word), 0.0))
        for word, _ in candidates
    ]

    top_words = [word for word, _ in sorted(scored, key=lambda x: -x[1])[:n_expand]]
    
    expanded_terms = []
    for word in top_words:
        hypernyms = get_wordnet_hypernym_synonyms(word, max_synonyms=max_synonyms, depth=depth)
        for hyp in hypernyms:
            if hyp in original_words:
                continue
            expanded_terms.append(hyp)
            if len(expanded_terms) >= max_synonyms:
                break

    return query_text + " " + " ".join(expanded_terms)

This script iterates over a list of queries, expands each query using hypernyms (with `expand_query_with_hypernyms`), and writes the expanded queries to a `.jsonl` file.

In [ ]:
expanded_queries_hyper = []
print("Expanding Queries with Hypernyms Only...")
for query in tqdm(queries, unit="query"):
    new_query = query.copy()
    new_query["expanded_text"] = expand_query_with_hypernyms(query["text"], vectorizer)
    expanded_queries_hyper.append(new_query)

with jsonlines.open("../data/trec-covid/queries_expanded_hypernyms.jsonl", mode='w') as writer:
    for q in expanded_queries_hyper:
        writer.write(q)
print("✅ Hypernym-expanded queries saved to ../data/trec-covid/queries_expanded_hypernyms.jsonl")

Expanding Queries with Hypernyms Only...


100%|██████████| 50/50 [00:32<00:00,  1.52query/s]

✅ Hypernym-expanded queries saved.


---

## 9. Retrieval with Expanded Queries

Load the expanded queries from disk, then for each cutoff \(k in \{20, 30, 50\}\) perform a `match` search on the `text` field using the `"expanded_text"`. Collect the top-\(k\) document IDs and their scores into separate JSON run files under `results/phase_2/`.

In [20]:
def process_queries_phase_2(expanded_queries_path, label='WordNet'):
    # Load queries
    with open(expanded_queries_path, 'r', encoding='utf-8') as f:
        queries = [json.loads(line) for line in f]

    INDEX_NAME = "ir2025-index"
    k_values = [20, 30, 50]

    runs = {f"run_{k}": {} for k in k_values}
    for k in k_values:
        output_dir = f"../results/phase_2"
        os.makedirs(output_dir, exist_ok=True)

        for query in tqdm(queries, desc=f"Processing Expanded Queries with {label} for run with k = {k}"):
            qid = query["_id"]
            query_text = query["expanded_text"] # This is the key the expanded query is saved under
            response = es.search(
                index=INDEX_NAME,
                query={"match": {"text": query_text}},
                size=k
            )
            # print(response)
            runs[f"run_{k}"][qid] = {hit["_id"]: hit["_score"] for hit in response["hits"]["hits"]}

        # Save each run
        with open(os.path.join(output_dir, f'retrieval_top_{k}.json'), 'w', encoding='utf-8') as f:
            json.dump(runs[f"run_{k}"], f, ensure_ascii=False, indent=4)
            print(f"✅ Results saved to: ../results/phase_2/retrieval_top_{k}_{label.lower()}.json")

    return runs
    
runs_wordnet = process_queries_phase_2("../data/trec-covid/queries_expanded_wordnet.jsonl")

Processing Expanded Queries with WordNet for run with k = 20:   0%|          | 0/50 [00:00<?, ?it/s]

Processing Expanded Queries with WordNet for run with k = 20: 100%|██████████| 50/50 [00:08<00:00,  6.20it/s]


✅ Results saved to: ../results/phase_2/retrieval_top_20_wordnet.json


Processing Expanded Queries with WordNet for run with k = 30: 100%|██████████| 50/50 [00:01<00:00, 39.28it/s]


✅ Results saved to: ../results/phase_2/retrieval_top_30_wordnet.json


Processing Expanded Queries with WordNet for run with k = 50: 100%|██████████| 50/50 [00:01<00:00, 34.49it/s]

✅ Results saved to: ../results/phase_2/retrieval_top_50_wordnet.json


In [21]:
runs_hypernyms = process_queries_phase_2("../data/trec-covid/queries_expanded_hypernyms.jsonl", label='Hypernyms')

Processing Expanded Queries with Hypernyms for run with k = 20:   0%|          | 0/50 [00:00<?, ?it/s]

Processing Expanded Queries with Hypernyms for run with k = 20: 100%|██████████| 50/50 [00:00<00:00, 80.55it/s]


✅ Results saved to: ../results/phase_2/retrieval_top_20_hypernyms.json


Processing Expanded Queries with Hypernyms for run with k = 30: 100%|██████████| 50/50 [00:00<00:00, 88.69it/s]


✅ Results saved to: ../results/phase_2/retrieval_top_30_hypernyms.json


Processing Expanded Queries with Hypernyms for run with k = 50: 100%|██████████| 50/50 [00:00<00:00, 81.11it/s]


✅ Results saved to: ../results/phase_2/retrieval_top_50_hypernyms.json


Read the TREC‐COVID qrels file into a nested dictionary of the form `{query_id: {doc_id: relevance_score}}`. Also compute and print the average number of relevant documents per query for insight into dataset sparsity.

In [22]:
def load_qrels(qrels_path="../data/trec-covid/qrels/test.tsv"):
    qrels = {}
    with open(qrels_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            qid = row['query-id']
            docid = row['corpus-id']
            relevance = int(row['score'])
            qrels.setdefault(qid, {})[docid] = relevance

    relevant_counts = Counter()
    for qid, docs in qrels.items():
        relevant_counts[qid] = sum(1 for rel in docs.values() if rel > 0)
    print("Average number of relevant documents per query:", int(sum(relevant_counts.values()) / len(relevant_counts)))

    return qrels

qrels = load_qrels()

Average number of relevant documents per query: 493


## 10. Evaluation of Expanded Runs with `pytrec_eval`

Use the `compute_metrics` function to evaluate each expanded-query run (`k` = 20, 30, 50) by:

- Computing **MAP** and **Precision@k** (for k = 5, 10, 15, 20) via `pytrec_eval.RelevanceEvaluator`  
- Saving **per-query** metrics to `results/phase_2/per_query_metrics_top_<k>.json`  
- Saving **average** metrics to `results/phase_2/average_metrics_top_<k>.json`

In [23]:
def compute_metrics(qrels, runs, folder, metrics=['map', 'P_5', 'P_10', 'P_15', 'P_20'], label="WordNet"):    
    # Metrics to Evaluate
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'P'})
    
    for run_name, run in runs.items():
        k = run_name.split("_")[1]
        print(f"Computing metrics for run with k = {k}")
        
        # Verify how many documents were retrieved per query
        # for query_id, docs in run.items():
        #     num_docs = len(docs)
        #     print(f"Query ID: {query_id} - Retrieved Documents: {num_docs}")
            
        results = evaluator.evaluate(run)
        
        #Print available metrics for debugging
        # first_query = list(results.keys())[0]
        # print(f"Available metrics for {first_query}: {list(results[first_query].keys())}")
        
        # Compute average metrics
        avg_scores = {metric: 0.0 for metric in metrics}
        num_queries = len(results)
        
        for res in results.values():
            for metric in metrics:
                avg_scores[metric] += res.get(metric, 0.0)
        
        for metric in metrics:
            avg_scores[metric] /= num_queries
                                                                                                                                               
        # Prepare output directory
        output_dir = os.path.join("../results", folder)
        os.makedirs(output_dir, exist_ok=True)
        
        # Save per-query metrics
        per_query_path = os.path.join(output_dir, f"per_query_metrics_top_{k}_{label.lower()}.json")
        with open(per_query_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4)
        
        # Save average metrics
        avg_metrics_path = os.path.join(output_dir, f"average_metrics_top_{k}_{label.lower()}.json")
        with open(avg_metrics_path, "w", encoding="utf-8") as f:
            json.dump(avg_scores, f, indent=4)
        
        print(f"✅ Per-query metrics saved to: {per_query_path}")
        print(f"✅ Average metrics saved to: {avg_metrics_path}\n")
        
compute_metrics(qrels, runs_wordnet, 'phase_2')

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_20_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_20_wordnet.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_30_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_30_wordnet.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_50_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_50_wordnet.json



In [24]:
compute_metrics(qrels, runs_hypernyms, 'phase_2', label="Hypernyms")

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_20_hypernyms.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_20_hypernyms.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_30_hypernyms.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_30_hypernyms.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_50_hypernyms.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_50_hypernyms.json



## 11. Compare Phase 1 & Phase 2 Metrics

Define a helper `compare_phases` to load average metrics for each phase (e.g. Phase 1 vs. Phase 2) at cutoffs k ∈ {20, 30, 50} and display a consolidated DataFrame:

- **Parameters**  
  - `phases`: dict mapping phase names to file path patterns, e.g.  
    ```python
    {
      "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
      "Phase 2": "../results/phase_2/average_metrics_top_{}.json"
    }
    ```  
  - `k_values`: list of cutoff values to compare (default `[20, 30, 50]`)  
  - `metrics`: list of metric keys to include (default `['map','P_5','P_10','P_15','P_20']`)  

- **Behavior**  
  1. For each k, loads the corresponding JSON file for each phase.  
  2. Extracts MAP and Precision@k metrics.  
  3. Builds a pandas DataFrame indexed by k.  
  4. Displays the comparison table side-by-side.

- **Returns**  
  - A `pandas.DataFrame` with rows for each k and columns for each phase’s MAP and avgPre@k.

In [25]:
def compare_phases(phases, k_values=[20, 30, 50], metrics=['map', 'P_5', 'P_10', 'P_15', 'P_20']):
    """
    Display and optionally compare retrieval metrics for 1 to 4 phases.
    Parameters:
    - phases: dict mapping phase names to base file paths, e.g.
        {
            "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
            "Phase 2": "../results/phase_2/average_metrics_top_{}.json",
            ...
        }
    - k_values: list of cutoff values to compare ([20, 30, 50])
    - metrics: list of TREC metric keys (['map', 'P_5', 'P_10'])

    Returns:
    - pandas DataFrame with metrics for all phases at each k
    """
    comparison = []

    for k in k_values:
        row = {"k": k}
        for phase_name, base_path in phases.items():
            try:
                with open(base_path.format(k), "r") as f:
                    phase_metrics = json.load(f)
                row[f"{phase_name} MAP"] = phase_metrics["map"]
                for m in metrics[1:]: # exclude MAP
                    row[f"{phase_name} avgPre@{m[2:]}"] = phase_metrics[m]
            except FileNotFoundError:
                print(f"⚠️ File not found: {base_path.format(k)}")
        comparison.append(row)

    df = pd.DataFrame(comparison)
    df.sort_values("k", inplace=True)
    df.set_index("k", inplace=True) # Set 'k' column as the index for visualization purposes
    display(df)
    return df

In [26]:
phases = {
    "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
    "Phase 2 - WordNet": "../results/phase_2/average_metrics_top_{}_wordnet.json",
    "Phase 2 - Hypernyms": "../results/phase_2/average_metrics_top_{}_hypernyms.json"
}
_ = compare_phases(phases)

,Phase 1 MAP,Phase 1 avgPre@5,Phase 1 avgPre@10,Phase 1 avgPre@15,Phase 1 avgPre@20,Phase 2 - WordNet MAP,Phase 2 - WordNet avgPre@5,Phase 2 - WordNet avgPre@10,Phase 2 - WordNet avgPre@15,Phase 2 - WordNet avgPre@20,Phase 2 - Hypernyms MAP,Phase 2 - Hypernyms avgPre@5,Phase 2 - Hypernyms avgPre@10,Phase 2 - Hypernyms avgPre@15,Phase 2 - Hypernyms avgPre@20
k,,,,,,,,,,,,,,,
20,0.020569,0.64,0.582,0.564,0.548,0.020554,0.608,0.586,0.556,0.538,0.020773,0.636,0.574,0.545333,0.537
30,0.027753,0.64,0.582,0.564,0.549,0.028373,0.608,0.586,0.556,0.538,0.028601,0.636,0.574,0.545333,0.537
50,0.039911,0.64,0.582,0.564,0.549,0.040848,0.608,0.586,0.556,0.538,0.040099,0.636,0.574,0.545333,0.537


| **k** | **Phase 1 MAP** | **Phase 1 avgPre\@5** | **Phase 1 avgPre\@10** | **Phase 1 avgPre\@15** | **Phase 1 avgPre\@20** |
| :---: | --------------: | --------------------: | ---------------------: | ---------------------: | ---------------------: |
|   20  |        0.020569 |                 0.640 |                  0.582 |                  0.564 |                  0.548 |
|   30  |        0.027753 |                 0.640 |                  0.582 |                  0.564 |                  0.549 |
|   50  |        0.039911 |                 0.640 |                  0.582 |                  0.564 |                  0.549 |


| **k** | **WordNet MAP** | **WordNet avgPre\@5** | **WordNet avgPre\@10** | **WordNet avgPre\@15** | **WordNet avgPre\@20** |
| :---: | --------------: | --------------------: | ---------------------: | ---------------------: | ---------------------: |
|   20  |        0.020554 |                 0.608 |                  0.586 |                  0.556 |                  0.538 |
|   30  |        0.028373 |                 0.608 |                  0.586 |                  0.556 |                  0.538 |
|   50  |        0.040848 |                 0.608 |                  0.586 |                  0.556 |                  0.538 |


| **k** | **Hypernyms MAP** | **Hypernyms avgPre\@5** | **Hypernyms avgPre\@10** | **Hypernyms avgPre\@15** | **Hypernyms avgPre\@20** |
| :---: | ----------------: | ----------------------: | -----------------------: | -----------------------: | -----------------------: |
|   20  |          0.020773 |                   0.636 |                    0.574 |                 0.545333 |                    0.537 |
|   30  |          0.028601 |                   0.636 |                    0.574 |                 0.545333 |                    0.537 |
|   50  |          0.040099 |                   0.636 |                    0.574 |                 0.545333 |                    0.537 |


| **k** | **Phase 1** | **Phase 2 A (WordNet)** |             **Δ A** | **Phase 2 B (Hypernyms)** |             **Δ B** |
| :---: | ----------: | ----------------------: | ------------------: | ------------------------: | ------------------: |
|   20  |    0.020569 |                0.020554 | −0.000015 (−0.07 %) |                  0.020773 | +0.000204 (+0.99 %) |
|   30  |    0.027753 |                0.028373 |  +0.000620 (+2.2 %) |                  0.028601 | +0.000848 (+3.06 %) |
|   50  |    0.039911 |                0.040848 | +0.000937 (+2.35 %) |                  0.040099 | +0.000188 (+0.47 %) |


|           | **Phase 1** | **Phase 2 A** |          **Δ A** | **Phase 2 B** |          **Δ B** |
| :-------: | ----------: | ------------: | ---------------: | ------------: | ---------------: |
| avgPre\@5 |       0.640 |         0.608 | −0.032 (−5.0 pp) |         0.636 | −0.004 (−0.6 pp) |


---

- Query Expansion **BEFORE** Query Preproccessing

In [7]:
with jsonlines.open('../data/trec-covid/corpus.jsonl') as reader:
    corpus = [obj for obj in reader]

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from tqdm import tqdm
import numpy as np
import joblib
import json
import os

def build_and_save_tfidf_model(corpus, output_dir="../models"):
    """
    Build TF-IDF model from corpus, compute statistics, and save the model.
    
    Args:
        corpus: List of documents with 'text' field
        output_dir: Directory to save models and statistics
    
    Returns:
        tuple: (vectorizer, idf_scores, statistics_dict)
    """
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    
    # Initialize counters for statistics
    all_tokens = []
    total_tokens = 0
    unique_tokens = set()
    statistics = {}

    # Preprocess with detailed statistics
    print("Preprocessing corpus...")
    preprocessed_corpus = []
    for doc in tqdm(corpus, desc="Preprocessing documents", unit="doc"):
        text = doc["text"]
        tokens = text.split()
        all_tokens.extend(tokens)

        # Update statistics
        total_tokens += len(tokens)
        unique_tokens.update(tokens)
        
        preprocessed_corpus.append(text)
    
    
    word_counts = Counter(all_tokens)
    global top_50_words
    top_50_words = [w for w, _ in word_counts.most_common(50)]
    print("Top 50 words:", top_50_words)

    # Save preprocessing statistics
    statistics['preprocessing'] = {
        'total_tokens': total_tokens,
        'unique_tokens': len(unique_tokens),
        'average_tokens_per_doc': total_tokens/len(corpus)
    }

    print(f"\nPreprocessing statistics:")
    print(f"- Total tokens: {total_tokens:,}")
    print(f"- Unique tokens: {len(unique_tokens):,}")
    print(f"- Average tokens per document: {total_tokens/len(corpus):,.1f}")

    # Build TF-IDF model with detailed progress
    print("\nBuilding TF-IDF model...")
    tfidf_vectorizer = TfidfVectorizer(lowercase=False, stop_words=None)

    with tqdm(total=3, desc="TF-IDF computation") as pbar:
        # Fit the vectorizer
        tfidf_vectorizer.fit(preprocessed_corpus)
        pbar.update(1)
        
        # Get feature names
        feature_names = tfidf_vectorizer.get_feature_names_out()
        pbar.update(1)
        
        # Calculate IDF scores
        idf_scores = dict(zip(feature_names, tfidf_vectorizer.idf_))
        pbar.update(1)

    # Calculate and save TF-IDF statistics
    idf_values = list(idf_scores.values())
    statistics['tfidf'] = {
        'vocabulary_size': len(idf_scores),
        'average_idf': float(np.mean(idf_values)),
        'max_idf': float(max(idf_values)),
        'min_idf': float(min(idf_values))
    }

    print("\nTF-IDF statistics:")
    print(f"- Vocabulary size: {len(idf_scores):,} terms")
    print(f"- Average IDF: {statistics['tfidf']['average_idf']:.2f}")
    print(f"- Max IDF: {statistics['tfidf']['max_idf']:.2f}")
    print(f"- Min IDF: {statistics['tfidf']['min_idf']:.2f}")

    # Save everything
    print("\nSaving models and statistics...")
    try:
        # Save vectorizer
        joblib.dump(tfidf_vectorizer, os.path.join(output_dir, 'tfidf_vectorizer_other.joblib'))
        
        # Save IDF scores
        with open(os.path.join(output_dir, 'idf_scores_other.json'), 'w', encoding='utf-8') as f:
            json.dump(idf_scores, f, ensure_ascii=False, indent=2)
            
        # Save statistics
        with open(os.path.join(output_dir, 'tfidf_statistics_other.json'), 'w', encoding='utf-8') as f:
            json.dump(statistics, f, ensure_ascii=False, indent=2)
            
        print("\n✅ Saved successfully:")
        print(f"- Vectorizer: {os.path.join(output_dir, 'tfidf_vectorizer_other.joblib')}")
        print(f"- IDF scores: {os.path.join(output_dir, 'idf_scores_other.json')}")
        print(f"- Statistics: {os.path.join(output_dir, 'tfidf_statistics_other.json')}")
        
    except Exception as e:
        print(f"\n❌ Error saving files: {e}")
        
    return tfidf_vectorizer, idf_scores

# Function to load the saved model
def load_tfidf_model(output_dir="../models"):
    """
    Load the saved TF-IDF model, IDF scores, and statistics.
    
    Args:
        output_dir: Directory where models and statistics are saved
        
    Returns:
        tuple: (vectorizer, idf_scores, statistics)
    """
    try:
        # Load vectorizer
        vectorizer = joblib.load(os.path.join(output_dir, 'tfidf_vectorizer_other.joblib'))
        
        # Load IDF scores
        with open(os.path.join(output_dir, 'idf_scores_other.json'), 'r', encoding='utf-8') as f:
            idf_scores = json.load(f)
            
        # Load statistics
        with open(os.path.join(output_dir, 'tfidf_statistics_other.json'), 'r', encoding='utf-8') as f:
            statistics = json.load(f)
        
        print("\nModel validation:")
        print(f"- Vocabulary size: {len(idf_scores):,}")
        print(f"- Average IDF: {statistics['tfidf']['average_idf']:.2f}")
        print(f"- Max IDF: {statistics['tfidf']['max_idf']:.2f}")
        print(f"- Min IDF: {statistics['tfidf']['min_idf']:.2f}")
 
        print("✅ Model loaded successfully")
        return vectorizer, idf_scores
        
    except Exception as e:
        print(f"❌ Error loading model: {e}")
        return None, None
        
# Transform new text using the loaded vectorizer
def transform_text(text, vectorizer):
    """Transform new text using the loaded vectorizer"""
    try:
        transformed = vectorizer.transform([text])
        return transformed
    except Exception as e:
        print(f"❌ Error transforming text: {e}")
        return None

In [9]:
# Test loading
vectorizer, idf_scores = load_tfidf_model()
if not vectorizer and not idf_scores:
    # Build and save the model
    vectorizer, idf_scores = build_and_save_tfidf_model(corpus)


Model validation:
- Vocabulary size: 241,243
- Average IDF: 11.57
- Max IDF: 12.36
- Min IDF: 1.31
✅ Model loaded successfully


In [13]:
from nltk.corpus import wordnet as wn
def get_wordnet_synonyms(word, max_synonyms=3):
    synonyms = set()
    word = word.lower()

    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            name = lemma.name().replace("_", " ").lower()

            # Filter out:
            if name == word: # Skip the original word
                continue
            if len(name.split()) > 1:  # Skip multi-word phrases
                continue
            if not name.isalpha(): # Skip non-alphabetic words
                continue

            synonyms.add(name)

    # Rank by frequency (most-used synonyms first)
    ranked_synonyms = sorted(synonyms, key=lambda s: -sum(lemma.count() for syn in wn.synsets(s) for lemma in syn.lemmas() if lemma.name().lower() == s))

    return ranked_synonyms[:max_synonyms] # Return up to max_synonyms

In [14]:
from nltk import pos_tag

def is_expandable(pos):
    return pos.startswith('NN') or pos.startswith('JJ')  # nouns & adjectives
    
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def expand_query_with_wordnet(query_text, tfidf_vectorizer, max_synonyms=1, n_expand=1):
    
    # For expansion decisions, work with the original query text
    tokens = word_tokenize(query_text)
    tagged = pos_tag(tokens)
    
    # Set for fast lookup
    original_words = set(tokens)
    
    # Candidate words = noun/adjective, not stopword, alphabetic
    candidates = [
        (word, pos) for word, pos in tagged
        if word.isalpha() and word not in stop_words and is_expandable(pos)
    ]

    # Get IDF scores from vectorizer to identify important terms
    # We need to preprocess the word to match vectorizer's vocabulary
    idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))
    
    # Score each candidate by IDF from the corpus
    scored = [
        (word, idf_scores.get(word, 0.0))
        for word, _ in candidates
    ]

    # Sort by IDF weight descending and keep top-n
    top_words = [word for word, _ in sorted(scored, key=lambda x: -x[1])[:n_expand]]
    
    # Expand only top words
    expanded_terms = []
    for word in top_words:
        synonyms = get_wordnet_synonyms(word, max_synonyms) # Get more, filter later
        for syn in synonyms:
            if syn in original_words:
                continue  # Skip if already in query
            expanded_terms.append(syn)
            if len(expanded_terms) >= max_synonyms:
                break  # limit per word

        # Return original query + expansion terms (let Elasticsearch handle preprocessing)
    return query_text + " " + " ".join(expanded_terms)

In [15]:
with jsonlines.open('../data/trec-covid/queries.jsonl') as reader:
    queries = [obj for obj in reader]
    print(f"Loaded {len(queries)} queries.")

Loaded 50 queries.


In [16]:
expanded_queries = []
print("Expanding Queries..")
for query in tqdm(queries, unit="query"):
    new_query = query.copy()
    new_query["expanded_text"] = expand_query_with_wordnet(query["text"], vectorizer)
    expanded_queries.append(new_query)

Expanding Queries..


100%|██████████| 50/50 [00:25<00:00,  1.94query/s]


In [17]:
with jsonlines.open("../data/trec-covid/queries_expanded_wordnet_other.jsonl", mode='w') as writer:
    for q in expanded_queries:
        writer.write(q)
    print("✅ Expanded queries saved to ../data/trec-covid/queries_expanded_wordnet_other.jsonl")

✅ Expanded queries saved to ../data/trec-covid/queries_expanded_wordnet_other.jsonl


In [18]:
def get_wordnet_hypernym_synonyms(word, max_synonyms=1, depth=1):
    """
    Get synonyms of hypernyms for a given word.
    
    Args:
        word: Target word.
        max_synonyms: Max number of terms to return.
        depth: How deep to go in the hypernym tree (1 = immediate parents).
        
    Returns:
        A ranked list of synonym candidates.
    """
    word = word.lower()
    synonyms = set()

    for syn in wn.synsets(word):
        # Explore hypernyms up to the given depth
        current_level = [syn]
        for _ in range(depth):
            next_level = []
            for s in current_level:
                next_level.extend(s.hypernyms())
            current_level = next_level

        # Collect lemma names from hypernyms
        for hyper in current_level:
            for lemma in hyper.lemmas():
                name = lemma.name().replace("_", " ").lower()
                if name != word and name.isalpha() and len(name.split()) == 1:
                    synonyms.add(name)

    # Rank by frequency
    ranked = sorted(synonyms, key=lambda s: -sum(lemma.count() for syn in wn.synsets(s) for lemma in syn.lemmas() if lemma.name().lower() == s))
    return ranked[:max_synonyms]

In [19]:
def expand_query_with_hypernyms(query_text, tfidf_vectorizer, max_synonyms=1, n_expand=1, depth=1):
    tokens = word_tokenize(query_text)
    tagged = pos_tag(tokens)
    original_words = set(tokens)

    candidates = [
        (word, pos) for word, pos in tagged
        if word.isalpha() and word not in stop_words and is_expandable(pos)
    ]

    idf_scores = dict(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_vectorizer.idf_))
    
    scored = [
        (word, idf_scores.get(word, 0.0))
        for word, _ in candidates
    ]

    top_words = [word for word, _ in sorted(scored, key=lambda x: -x[1])[:n_expand]]
    
    expanded_terms = []
    for word in top_words:
        hypernyms = get_wordnet_hypernym_synonyms(word, max_synonyms=max_synonyms, depth=depth)
        for hyp in hypernyms:
            if hyp in original_words:
                continue
            expanded_terms.append(hyp)
            if len(expanded_terms) >= max_synonyms:
                break

    return query_text + " " + " ".join(expanded_terms)

In [20]:
expanded_queries_hyper = []
print("Expanding Queries with Hypernyms Only...")
for query in tqdm(queries, unit="query"):
    new_query = query.copy()
    new_query["expanded_text"] = expand_query_with_hypernyms(query["text"], vectorizer)
    expanded_queries_hyper.append(new_query)

with jsonlines.open("../data/trec-covid/queries_expanded_hypernyms_other.jsonl", mode='w') as writer:
    for q in expanded_queries_hyper:
        writer.write(q)
print("✅ Hypernym-expanded queries saved to ../data/trec-covid/queries_expanded_hypernyms_other.jsonl")

Expanding Queries with Hypernyms Only...


100%|██████████| 50/50 [00:23<00:00,  2.09query/s]

✅ Hypernym-expanded queries saved to ../data/trec-covid/queries_expanded_hypernyms_other.jsonl


In [21]:
def load_qrels(qrels_path="../data/trec-covid/qrels/test.tsv"):
    qrels = {}
    with open(qrels_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f, delimiter='\t')
        for row in reader:
            qid = row['query-id']
            docid = row['corpus-id']
            relevance = int(row['score'])
            qrels.setdefault(qid, {})[docid] = relevance

    relevant_counts = Counter()
    for qid, docs in qrels.items():
        relevant_counts[qid] = sum(1 for rel in docs.values() if rel > 0)
    print("Average number of relevant documents per query:", int(sum(relevant_counts.values()) / len(relevant_counts)))

    return qrels

qrels = load_qrels()

Average number of relevant documents per query: 493


In [22]:
def process_queries_phase_22(expanded_queries_path, label='WordNet'):
    # Load queries
    with open(expanded_queries_path, 'r', encoding='utf-8') as f:
        queries = [json.loads(line) for line in f]

    INDEX_NAME = "ir2025-index"
    k_values = [20, 30, 50]

    runs = {f"run_{k}": {} for k in k_values}
    for k in k_values:
        output_dir = f"../results/phase_22"
        os.makedirs(output_dir, exist_ok=True)

        for query in tqdm(queries, desc=f"Processing Expanded Queries with {label} for run with k = {k}"):
            qid = query["_id"]
            query_text = query["expanded_text"] # This is the key the expanded query is saved under
            response = es.search(
                index=INDEX_NAME,
                query={"match": {"text": query_text}},
                size=k
            )
            # print(response)
            runs[f"run_{k}"][qid] = {hit["_id"]: hit["_score"] for hit in response["hits"]["hits"]}

        # Save each run
        with open(os.path.join(output_dir, f'retrieval_top_{k}.json'), 'w', encoding='utf-8') as f:
            json.dump(runs[f"run_{k}"], f, ensure_ascii=False, indent=4)
            print(f"✅ Results saved to: ../results/phase_22/retrieval_top_{k}_{label.lower()}.json")

    return runs

In [23]:
runs_wordnet = process_queries_phase_22("../data/trec-covid/queries_expanded_wordnet_other.jsonl")

Processing Expanded Queries with WordNet for run with k = 20: 100%|██████████| 50/50 [00:03<00:00, 13.79it/s]


✅ Results saved to: ../results/phase_22/retrieval_top_20_wordnet.json


Processing Expanded Queries with WordNet for run with k = 30: 100%|██████████| 50/50 [00:02<00:00, 20.85it/s]


✅ Results saved to: ../results/phase_22/retrieval_top_30_wordnet.json


Processing Expanded Queries with WordNet for run with k = 50: 100%|██████████| 50/50 [00:01<00:00, 28.26it/s]

✅ Results saved to: ../results/phase_22/retrieval_top_50_wordnet.json


In [ ]:
def compute_metrics(qrels, runs, folder, metrics=['map', 'P_5', 'P_10', 'P_15', 'P_20'], label="WordNet"):    
    # Metrics to Evaluate
    evaluator = pytrec_eval.RelevanceEvaluator(qrels, {'map', 'P'})
    
    for run_name, run in runs.items():
        k = run_name.split("_")[1]
        print(f"Computing metrics for run with k = {k}")
        
        # Verify how many documents were retrieved per query
        # for query_id, docs in run.items():
        #     num_docs = len(docs)
        #     print(f"Query ID: {query_id} - Retrieved Documents: {num_docs}")
            
        results = evaluator.evaluate(run)
        
        #Print available metrics for debugging
        # first_query = list(results.keys())[0]
        # print(f"Available metrics for {first_query}: {list(results[first_query].keys())}")
        
        # Compute average metrics
        avg_scores = {metric: 0.0 for metric in metrics}
        num_queries = len(results)
        
        for res in results.values():
            for metric in metrics:
                avg_scores[metric] += res.get(metric, 0.0)
        
        for metric in metrics:
            avg_scores[metric] /= num_queries
                                                                                                                                               
        # Prepare output directory
        output_dir = os.path.join("../results", folder)
        os.makedirs(output_dir, exist_ok=True)
        
        # Save per-query metrics
        per_query_path = os.path.join(output_dir, f"per_query_metrics_top_{k}_{label.lower()}.json")
        with open(per_query_path, "w", encoding="utf-8") as f:
            json.dump(results, f, indent=4)
        
        # Save average metrics
        avg_metrics_path = os.path.join(output_dir, f"average_metrics_top_{k}_{label.lower()}.json")
        with open(avg_metrics_path, "w", encoding="utf-8") as f:
            json.dump(avg_scores, f, indent=4)
        
        print(f"✅ Per-query metrics saved to: {per_query_path}")
        print(f"✅ Average metrics saved to: {avg_metrics_path}\n")

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_20_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_20_wordnet.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_30_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_30_wordnet.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_2\per_query_metrics_top_50_wordnet.json
✅ Average metrics saved to: ../results\phase_2\average_metrics_top_50_wordnet.json



In [32]:
compute_metrics(qrels, runs_wordnet, 'phase_22')

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_20_wordnet.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_20_wordnet.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_30_wordnet.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_30_wordnet.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_50_wordnet.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_50_wordnet.json



In [28]:
runs_hypernyms = process_queries_phase_22("../data/trec-covid/queries_expanded_hypernyms_other.jsonl", label='Hypernyms')

Processing Expanded Queries with Hypernyms for run with k = 20: 100%|██████████| 50/50 [00:00<00:00, 52.60it/s]


✅ Results saved to: ../results/phase_22/retrieval_top_20_hypernyms.json


Processing Expanded Queries with Hypernyms for run with k = 30: 100%|██████████| 50/50 [00:00<00:00, 55.78it/s]


✅ Results saved to: ../results/phase_22/retrieval_top_30_hypernyms.json


Processing Expanded Queries with Hypernyms for run with k = 50: 100%|██████████| 50/50 [00:01<00:00, 46.94it/s]

✅ Results saved to: ../results/phase_22/retrieval_top_50_hypernyms.json


In [33]:
compute_metrics(qrels, runs_hypernyms, 'phase_22', label="Hypernyms")

Computing metrics for run with k = 20
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_20_hypernyms.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_20_hypernyms.json

Computing metrics for run with k = 30
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_30_hypernyms.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_30_hypernyms.json

Computing metrics for run with k = 50
✅ Per-query metrics saved to: ../results\phase_22\per_query_metrics_top_50_hypernyms.json
✅ Average metrics saved to: ../results\phase_22\average_metrics_top_50_hypernyms.json



In [36]:
phases = {
    "Phase 2 - WordNet": "../results/phase_2/average_metrics_top_{}_wordnet.json",
    "Phase 22 - WordNet": "../results/phase_22/average_metrics_top_{}_wordnet.json",
}
compare_phases(phases)

,Phase 2 - WordNet MAP,Phase 2 - WordNet avgPre@5,Phase 2 - WordNet avgPre@10,Phase 2 - WordNet avgPre@15,Phase 2 - WordNet avgPre@20,Phase 22 - WordNet MAP,Phase 22 - WordNet avgPre@5,Phase 22 - WordNet avgPre@10,Phase 22 - WordNet avgPre@15,Phase 22 - WordNet avgPre@20
k,,,,,,,,,,
20,0.020554,0.608,0.586,0.556,0.538,0.020366,0.596,0.588,0.552,0.538
30,0.028373,0.608,0.586,0.556,0.538,0.028521,0.596,0.588,0.552,0.538
50,0.040848,0.608,0.586,0.556,0.538,0.040664,0.596,0.588,0.552,0.538


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Phase 2 - WordNet MAP</th>
      <th>Phase 2 - WordNet avgPre@5</th>
      <th>Phase 2 - WordNet avgPre@10</th>
      <th>Phase 2 - WordNet avgPre@15</th>
      <th>Phase 2 - WordNet avgPre@20</th>
      <th>Phase 22 - WordNet MAP</th>
      <th>Phase 22 - WordNet avgPre@5</th>
      <th>Phase 22 - WordNet avgPre@10</th>
      <th>Phase 22 - WordNet avgPre@15</th>
      <th>Phase 22 - WordNet avgPre@20</th>
    </tr>
    <tr>
      <th>k</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>20</th>
      <td>0.020554</td>
      <td>0.608</td>
      <td>0.586</td>
      <td>0.556</td>
      <td>0.538</td>
      <td>0.020366</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
    </tr>
    <tr>
      <th>30</th>
      <td>0.028373</td>
      <td>0.608</td>
      <td>0.586</td>
      <td>0.556</td>
      <td>0.538</td>
      <td>0.028521</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
    </tr>
    <tr>
      <th>50</th>
      <td>0.040848</td>
      <td>0.608</td>
      <td>0.586</td>
      <td>0.556</td>
      <td>0.538</td>
      <td>0.040664</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
    </tr>
  </tbody>
</table>
</div>

| **k** | **MAP**<br>(Preprocess → Expand) | **MAP**<br>(Expand → Preprocess) | **Δ MAP** | **avgPre@5**       | **avgPre@10**      | **avgPre@15**      | **avgPre@20**      |
|----------------|----------------------------------|----------------------------------|-----------|---------------------|---------------------|---------------------|---------------------|
| **20**         | 0.020554                         | 0.020366                         | −0.000188 | 0.608 → **0.596**   | 0.586 → **0.588**   | 0.556 → **0.552**   | 0.538 → **0.538**   |
| **30**         | 0.028373                         | 0.028521                         | +0.000148 | 0.608 → **0.596**   | 0.586 → **0.588**   | 0.556 → **0.552**   | 0.538 → **0.538**   |
| **50**         | 0.040848                         | 0.040664                         | −0.000184 | 0.608 → **0.596**   | 0.586 → **0.588**   | 0.556 → **0.552**   | 0.538 → **0.538**   |


In [37]:
phases = {
    "Phase 2 - Hypernyms": "../results/phase_2/average_metrics_top_{}_hypernyms.json",
    "Phase 22 - Hypernyms": "../results/phase_22/average_metrics_top_{}_hypernyms.json"
}
compare_phases(phases)

,Phase 2 - Hypernyms MAP,Phase 2 - Hypernyms avgPre@5,Phase 2 - Hypernyms avgPre@10,Phase 2 - Hypernyms avgPre@15,Phase 2 - Hypernyms avgPre@20,Phase 22 - Hypernyms MAP,Phase 22 - Hypernyms avgPre@5,Phase 22 - Hypernyms avgPre@10,Phase 22 - Hypernyms avgPre@15,Phase 22 - Hypernyms avgPre@20
k,,,,,,,,,,
20,0.020773,0.636,0.574,0.545333,0.537,0.020761,0.64,0.576,0.546667,0.534
30,0.028601,0.636,0.574,0.545333,0.537,0.028542,0.64,0.576,0.546667,0.534
50,0.040099,0.636,0.574,0.545333,0.537,0.040006,0.64,0.576,0.546667,0.534


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Phase 2 - Hypernyms MAP</th>
      <th>Phase 2 - Hypernyms avgPre@5</th>
      <th>Phase 2 - Hypernyms avgPre@10</th>
      <th>Phase 2 - Hypernyms avgPre@15</th>
      <th>Phase 2 - Hypernyms avgPre@20</th>
      <th>Phase 22 - Hypernyms MAP</th>
      <th>Phase 22 - Hypernyms avgPre@5</th>
      <th>Phase 22 - Hypernyms avgPre@10</th>
      <th>Phase 22 - Hypernyms avgPre@15</th>
      <th>Phase 22 - Hypernyms avgPre@20</th>
    </tr>
    <tr>
      <th>k</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>20</th>
      <td>0.020773</td>
      <td>0.636</td>
      <td>0.574</td>
      <td>0.545333</td>
      <td>0.537</td>
      <td>0.020761</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
    <tr>
      <th>30</th>
      <td>0.028601</td>
      <td>0.636</td>
      <td>0.574</td>
      <td>0.545333</td>
      <td>0.537</td>
      <td>0.028542</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
    <tr>
      <th>50</th>
      <td>0.040099</td>
      <td>0.636</td>
      <td>0.574</td>
      <td>0.545333</td>
      <td>0.537</td>
      <td>0.040006</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
  </tbody>
</table>
</div>

| **Cutoff (k)** | **MAP**<br>(Preprocess → Expand) | **MAP**<br>(Expand → Preprocess) | **Δ MAP** | **avgPre@5**       | **avgPre@10**      | **avgPre@15**      | **avgPre@20**      |
|----------------|----------------------------------|----------------------------------|-----------|---------------------|---------------------|---------------------|---------------------|
| **20**         | 0.020773                         | 0.020761                         | −0.000012 | 0.636 → **0.640**   | 0.574 → **0.576**   | 0.545 → **0.547**   | 0.537 → **0.534**   |
| **30**         | 0.028601                         | 0.028542                         | −0.000059 | 0.636 → **0.640**   | 0.574 → **0.576**   | 0.545 → **0.547**   | 0.537 → **0.534**   |
| **50**         | 0.040099                         | 0.040006                         | −0.000093 | 0.636 → **0.640**   | 0.574 → **0.576**   | 0.545 → **0.547**   | 0.537 → **0.534**   |


In [35]:
phases = {
    "Phase 1": "../results/phase_1/average_metrics_top_{}.json",
    "Phase 22 - WordNet": "../results/phase_22/average_metrics_top_{}_wordnet.json",
    "Phase 22 - Hypernyms": "../results/phase_22/average_metrics_top_{}_hypernyms.json"
}
compare_phases(phases)

,Phase 1 MAP,Phase 1 avgPre@5,Phase 1 avgPre@10,Phase 1 avgPre@15,Phase 1 avgPre@20,Phase 22 - WordNet MAP,Phase 22 - WordNet avgPre@5,Phase 22 - WordNet avgPre@10,Phase 22 - WordNet avgPre@15,Phase 22 - WordNet avgPre@20,Phase 22 - Hypernyms MAP,Phase 22 - Hypernyms avgPre@5,Phase 22 - Hypernyms avgPre@10,Phase 22 - Hypernyms avgPre@15,Phase 22 - Hypernyms avgPre@20
k,,,,,,,,,,,,,,,
20,0.020569,0.64,0.582,0.564,0.548,0.020366,0.596,0.588,0.552,0.538,0.020761,0.64,0.576,0.546667,0.534
30,0.027753,0.64,0.582,0.564,0.549,0.028521,0.596,0.588,0.552,0.538,0.028542,0.64,0.576,0.546667,0.534
50,0.039911,0.64,0.582,0.564,0.549,0.040664,0.596,0.588,0.552,0.538,0.040006,0.64,0.576,0.546667,0.534


<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Phase 1 MAP</th>
      <th>Phase 1 avgPre@5</th>
      <th>Phase 1 avgPre@10</th>
      <th>Phase 1 avgPre@15</th>
      <th>Phase 1 avgPre@20</th>
      <th>Phase 22 - WordNet MAP</th>
      <th>Phase 22 - WordNet avgPre@5</th>
      <th>Phase 22 - WordNet avgPre@10</th>
      <th>Phase 22 - WordNet avgPre@15</th>
      <th>Phase 22 - WordNet avgPre@20</th>
      <th>Phase 22 - Hypernyms MAP</th>
      <th>Phase 22 - Hypernyms avgPre@5</th>
      <th>Phase 22 - Hypernyms avgPre@10</th>
      <th>Phase 22 - Hypernyms avgPre@15</th>
      <th>Phase 22 - Hypernyms avgPre@20</th>
    </tr>
    <tr>
      <th>k</th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>20</th>
      <td>0.020569</td>
      <td>0.64</td>
      <td>0.582</td>
      <td>0.564</td>
      <td>0.548</td>
      <td>0.020366</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
      <td>0.020761</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
    <tr>
      <th>30</th>
      <td>0.027753</td>
      <td>0.64</td>
      <td>0.582</td>
      <td>0.564</td>
      <td>0.549</td>
      <td>0.028521</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
      <td>0.028542</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
    <tr>
      <th>50</th>
      <td>0.039911</td>
      <td>0.64</td>
      <td>0.582</td>
      <td>0.564</td>
      <td>0.549</td>
      <td>0.040664</td>
      <td>0.596</td>
      <td>0.588</td>
      <td>0.552</td>
      <td>0.538</td>
      <td>0.040006</td>
      <td>0.64</td>
      <td>0.576</td>
      <td>0.546667</td>
      <td>0.534</td>
    </tr>
  </tbody>
</table>
</div>